<a href="https://colab.research.google.com/github/mmikite/MI_hzxa37/blob/main/6_megoldas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neurális hálózatok - Feladatok

Pajzsmirigybetegség osztályozása

Adatok

Pajzsmirigybetegség (ann-thyroid) adathalmaz az UCI gépi tanulási tárából (*A Thyroid database suited for training ANNs*), amely alkalmas neurális hálózatok kipróbálására.
3772 tanulási adatot tartalmaz. 15 kategorikus és 6 valós attribútummal rendelkezik.
A probléma annak megállapítása, hogy a klinikára beutalt beteg pajzsmirigy alulműködésű-e. Két osztályt képezünk: normális/beteg; ezt a 21. oszlop tartalmazza.

Beolvassuk a szükséges könyvtárakat.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# for splitting of data
from sklearn.model_selection import train_test_split

# to get optimal hyperparameters' values
from sklearn.model_selection import GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



Beolvassuk a adatokat


In [ ]:
filepath = "http://web.uni-corvinus.hu/~fszabina/data/annthyroid-training.csv"

df = pd.read_csv(filepath, header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.750000,1,0,1,1,1,1,1,0,1,...,1,1,1,1,0.001132,0.080780,0.197324,0.300926,0.225000,1
1,0.239583,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0.000472,0.164345,0.235786,0.537037,0.165625,1
2,0.479167,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0.003585,0.130919,0.167224,0.527778,0.118750,1
3,0.656250,0,1,1,1,1,1,1,1,1,...,1,1,1,1,0.001698,0.091922,0.125418,0.337963,0.129688,1
4,0.229167,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0.000472,0.142061,0.229097,0.337963,0.235938,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0.875000,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0.060377,0.050696,0.088629,0.333333,0.093750,-1
6996,0.218750,0,1,1,1,1,1,1,1,1,...,1,1,1,1,0.004340,0.097493,0.239130,0.347222,0.243750,1
6997,0.229167,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0.005094,0.109192,0.103679,0.291667,0.121875,1
6998,0.531250,0,1,1,1,1,1,1,1,1,...,1,1,1,0,0.002830,0.109192,0.160535,0.328704,0.170313,1


Ellenőrizzük a hiányzó értékeket az adatállományban.

In [ ]:
if df.isnull().sum().sum()==0:
    print("No missing data")

else:
    print("Dataset with missing data")
    print(df.isnull().sum())

No missing data


X és y meghatározása a dataframe-ből, és ezen adatok felosztása a képzési halmazra és az érvényesítési halmazra.

In [ ]:
X = df.iloc[:,:-1]
y = df[21]

# splitting training and validation data
train_X, valid_X, train_y, valid_y = train_test_split(X , y, train_size = 0.8, test_size = 0.2, random_state = 0)

F1 érték elérése a modellek számára.

In [ ]:
def getF1(tn, fp, fn, tp):
    accuracy = (tn+tp)/(tn+fn+fp+tp)

    # sensitivity/recall - ratio between number of true positives and
    # the number of all samples whose class is the positive one
    sensitivity = tp/(tp+fn)

    # specificity - true negative (correct negative predictions
    # to actual negatives)
    specificity = tn/(tn+fp)

    # precision - ratio between number of true positives and
    # number of all samples classified as positive
    precision = tp/(tp+fp)

    f1 = (2*(precision*sensitivity))/(precision+sensitivity)

    return f1

# K Nearest Neighbors

In [ ]:
model = KNeighborsClassifier()
model.fit(train_X, train_y)
predictions = model.predict(valid_X)

tn, fp, fn, tp = confusion_matrix(valid_y.values, predictions).ravel()
print("True Negative: {}".format(tn))
print("False Positive: {}".format(fp))
print("False Negative: {}".format(fn))
print("True Positive: {}".format(tp))

res_KNN = getF1(tn, fp, fn, tp)
print(res_KNN)
#dict_results['K Nearest Neighbors'] = res_KNN

True Negative: 34
False Positive: 74
False Negative: 6
True Positive: 1286
0.9698340874811462


# Multilayer Perception

In [ ]:
model = MLPClassifier(random_state=0, max_iter=10000)
# also tried a higher max iter value but f1 value did not increase
# max iter fixed warning message

model.fit(train_X, train_y)
predictions = model.predict(valid_X)

tn, fp, fn, tp = confusion_matrix(valid_y.values, predictions).ravel()
print("True Negative: {}".format(tn))
print("False Positive: {}".format(fp))
print("False Negative: {}".format(fn))
print("True Positive: {}".format(tp))

res_MLP = getF1(tn, fp, fn, tp)
print(res_MLP)
#dict_results['Multilayer Perception'] = res_MLP

True Negative: 102
False Positive: 6
False Negative: 10
True Positive: 1282
0.9937984496124032
